In [2]:
!pip install groq python-dotenv numpy tqdm datasets

In [3]:
from groq import Groq
from dotenv import load_dotenv
from datasets import load_dataset

import os
from tqdm import tqdm
import re
import random
import pprint

from typing import List, Dict, Any

load_dotenv()
random.seed(0)

client = Groq()
gsm8k_dataset = load_dataset("gsm8k", "main")

gsm8k_train = gsm8k_dataset["train"]
gsm8k_test  = gsm8k_dataset["test"]

In [4]:
def generate_response_using_Llama(
        prompt: str,
        model: str = "llama-3.1-8b-instant"
    ):
    try:
        chat_completion = client.chat.completions.create(
            messages=[
                {
                    "role": "system",
                    "content": "You are a helpful assistant that solves math problems."
                },
                {
                    "role": "user", 
                    "content": prompt
                }
            ],
            model=model,
            temperature=0.3, ### 수정해도 됩니다!
            stream=False
        )
        return chat_completion.choices[0].message.content
    
    except Exception as e:
        print(f"API call error: {str(e)}")
        return None

#### 응답 잘 나오는지 확인해보기

In [5]:
response = generate_response_using_Llama(
    prompt="Hello world!",
)
print(response)

Hello! It's nice to meet you. I'm here to help with any math problems you might have. What's on your mind? Do you have a specific problem you'd like help with, or do you want to practice some math concepts?


#### GSM8K 데이터셋 확인해보기

In [6]:
print("[Question]")
for l in gsm8k_test['question'][0].split("."):
    print(l)
print("="*100)
print("[Answer]")
print(gsm8k_test['answer'][0])

[Question]
Janet’s ducks lay 16 eggs per day
 She eats three for breakfast every morning and bakes muffins for her friends every day with four
 She sells the remainder at the farmers' market daily for $2 per fresh duck egg
 How much in dollars does she make every day at the farmers' market?
[Answer]
Janet sells 16 - 3 - 4 = <<16-3-4=9>>9 duck eggs a day.
She makes 9 * 2 = $<<9*2=18>>18 every day at the farmer’s market.
#### 18


#### Util 함수들
- extract_final_answer: LLM의 응답을 parse하여 최종 결과만 추출 (정답과 비교하기 위해)
- run_benchmark_test: 벤치마크 테스트
- save_final_result: 결과물 제출을 위한 함수

In [7]:
### 수정해도 됩니다!
def extract_final_answer(response: str):
    regex = r"(?:Answer:|Model response:)\s*\$?([0-9,]+)\b|([0-9,]+)\s*(meters|cups|miles|minutes)"
    matches = re.finditer(regex, response, re.MULTILINE)
    results = [match.group(1) if match.group(1) else match.group(2).replace(",", "") for match in matches]

    if len(results) == 0:
        additional_regex = r"\$?([0-9,]+)"
        additional_matches = re.finditer(additional_regex, response, re.MULTILINE)
        results.extend([match.group(1).replace(",", "") for match in additional_matches])

    return results[-1] if results else None

In [8]:
### 수정해도 됩니다!
def run_benchmark_test(
        dataset,
        prompt: str,
        model: str = "llama-3.1-8b-instant",
        num_samples: int = 50,
        VERBOSE: bool = False
    ):
    correct = 0
    total   = 0
    results = []

    for i in tqdm(range(min(num_samples, len(dataset)))):
        question = dataset[i]["question"]
        correct_answer = float(re.findall(r'\d+(?:\.\d+)?', dataset[i]["answer"].split('####')[-1])[0])

        response = generate_response_using_Llama(
            prompt=prompt.format(question=question),
            model=model
        )

        if response:
            if VERBOSE:
                print("="*50)
                print(response)
                print("="*50)
            predicted_answer = extract_final_answer(response)

            if isinstance(predicted_answer, str):
                if predicted_answer.strip() == '':
                    predicted_answer = None
                else:
                    try:
                        predicted_answer = float(predicted_answer.replace(",", ""))
                    except ValueError:
                        predicted_answer = None
            
            diff = abs(predicted_answer - correct_answer) if predicted_answer is not None else float('inf')
            is_correct = diff < 1e-5 if predicted_answer is not None else False
            
            if is_correct:
                correct += 1
            total += 1
            
            results.append({
                'question': question,
                'correct_answer': correct_answer,
                'predicted_answer': predicted_answer,
                'response': response,
                'correct': is_correct
            })

            if (i + 1) % 5 == 0:
                current_acc = correct/total if total > 0 else 0
                print(f"Progress: [{i+1}/{num_samples}]")
                print(f"Current Acc.: [{current_acc:.2%}]")

    return results, correct/total if total > 0 else 0

In [9]:
def save_final_result(results: List[Dict[str, Any]], accuracy: float, filename: str) -> None:
    result_str = f"====== ACCURACY: {accuracy} ======\n\n"
    result_str += f"[Details]\n"
    
    for idx, result in enumerate(results):
        result_str += f"Question {idx+1}: {result['question']}\n"
        result_str += f"Correct Answer: {result['correct_answer']}\n"
        result_str += f"Predicted Answer: {result['predicted_answer']}\n"
        result_str += f"Correct: {result['correct']}\n\n"
    
    with open(filename, "w", encoding="utf-8") as f:
        f.write(result_str)

#### Direct prompting with few-shot example

In [15]:
def construct_direct_prompt(num_examples: int = 3) -> str:
    train_dataset = gsm8k_train

    sampled_indices = random.sample(
        [i for i in range(len(train_dataset['question']))],
        num_examples
    )

    prompt = "Instruction:\nSolve the following mathematical question and generate ONLY the answer after a tag, 'Answer:' without any rationale.\n"

    for i in range(num_examples):
        cur_question = train_dataset['question'][i]
        cur_answer = train_dataset['answer'][i].split("####")[-1].strip()

        prompt += f"\n[Example {i+1}]\n"
        prompt += f"Question:\n{cur_question}\n"
        prompt += f"Answer:{cur_answer}\n"

    prompt += "\nQuestion:\n{question}\nAnswer:"

    return prompt

In [16]:
### 어떤 방식으로 저장되는지 확인해보세요!
PROMPT = construct_direct_prompt(3)
VERBOSE = False

results, accuracy = run_benchmark_test(
    dataset=gsm8k_test,
    prompt=PROMPT,
    VERBOSE=VERBOSE,
    num_samples=10
)
save_final_result(results, accuracy, "example.txt")

 50%|█████     | 5/10 [00:02<00:02,  1.84it/s]

Progress: [5/10]
Current Acc.: [60.00%]


100%|██████████| 10/10 [00:04<00:00,  2.26it/s]

Progress: [10/10]
Current Acc.: [50.00%]


In [17]:
PROMPT = construct_direct_prompt(0)
VERBOSE = False

results, accuracy = run_benchmark_test(
    dataset=gsm8k_test,
    prompt=PROMPT,
    VERBOSE=VERBOSE,
    num_samples=50
)
save_final_result(results, accuracy, "direct_prompting_0.txt")

 10%|█         | 5/50 [00:01<00:15,  2.88it/s]

Progress: [5/50]
Current Acc.: [80.00%]


 20%|██        | 10/50 [00:03<00:16,  2.42it/s]

Progress: [10/50]
Current Acc.: [60.00%]


 30%|███       | 15/50 [00:14<01:14,  2.12s/it]

...

Progress: [40/50]
Current Acc.: [77.50%]


 90%|█████████ | 45/50 [01:22<00:13,  2.77s/it]

Progress: [45/50]
Current Acc.: [75.56%]


100%|██████████| 50/50 [01:35<00:00,  1.91s/it]

Progress: [50/50]
Current Acc.: [76.00%]


In [18]:
PROMPT = construct_direct_prompt(3)
VERBOSE = False

results, accuracy = run_benchmark_test(
    dataset=gsm8k_test,
    prompt=PROMPT,
    VERBOSE=VERBOSE,
    num_samples=50
)
save_final_result(results, accuracy, "direct_prompting_3.txt")

 10%|█         | 5/50 [00:03<00:42,  1.06it/s]

Progress: [5/50]
Current Acc.: [80.00%]


 20%|██        | 10/50 [00:23<02:24,  3.60s/it]

Progress: [10/50]
Current Acc.: [60.00%]


 30%|███       | 15/50 [00:50<02:57,  5.06s/it]

...

Progress: [40/50]
Current Acc.: [75.00%]


 90%|█████████ | 45/50 [02:49<00:19,  3.91s/it]

Progress: [45/50]
Current Acc.: [73.33%]


100%|██████████| 50/50 [03:08<00:00,  3.78s/it]

Progress: [50/50]
Current Acc.: [74.00%]


In [19]:
PROMPT = construct_direct_prompt(5)
VERBOSE = False

results, accuracy = run_benchmark_test(
    dataset=gsm8k_test,
    prompt=PROMPT,
    VERBOSE=VERBOSE,
    num_samples=50
)
save_final_result(results, accuracy, "direct_prompting_5.txt")

 10%|█         | 5/50 [00:20<03:44,  4.98s/it]

Progress: [5/50]
Current Acc.: [100.00%]


 20%|██        | 10/50 [00:44<03:16,  4.90s/it]

Progress: [10/50]
Current Acc.: [80.00%]


 30%|███       | 15/50 [01:07<02:43,  4.67s/it]

...

Progress: [40/50]
Current Acc.: [80.00%]


 90%|█████████ | 45/50 [03:37<00:23,  4.75s/it]

Progress: [45/50]
Current Acc.: [77.78%]


100%|██████████| 50/50 [04:02<00:00,  4.85s/it]

Progress: [50/50]
Current Acc.: [78.00%]


### Chain-of-Thought prompting with few-shot example
```text
[Question]
Janet’s ducks lay 16 eggs per day
 She eats three for breakfast every morning and bakes muffins for her friends every day with four
 She sells the remainder at the farmers' market daily for $2 per fresh duck egg
 How much in dollars does she make every day at the farmers' market?
====================================================================================================
[Answer]
Janet sells 16 - 3 - 4 = <<16-3-4=9>>9 duck eggs a day.
She makes 9 * 2 = $<<9*2=18>>18 every day at the farmer’s market.
#### 18
```

[Answer] 아래의 정답을 도출하는 과정을 예시로 달아주면 CoT의 few shot이 되겠죠?

In [20]:
def construct_CoT_prompt(num_examples: int = 3) -> str:
    train_dataset = gsm8k_train

    sampled_indices = random.sample(
        [i for i in range(len(train_dataset['question']))],
        num_examples
    )
    prompt = "Solve the following math problems step by step clearly."

    for i in range(num_examples):
        idx = sampled_indices[i]
        question = train_dataset['question'][idx]
        answer = train_dataset['answer'][idx]
        
        prompt += "[Question]\n"
        prompt += f"{question}\n"
        prompt += "=" * 100 + "\n"
        prompt += "[Answer]\n"
        prompt += f"{answer}\n\n"
        pass

    prompt += "\nQuestion:\n{question}\nAnswer:"

    return prompt

In [21]:
PROMPT_0 = construct_CoT_prompt(0)
results_0, accuracy_0 = run_benchmark_test(
    dataset=gsm8k_test,
    prompt=PROMPT_0,
    VERBOSE=False,
    num_samples=50
)
save_final_result(results_0, accuracy_0, "CoT_prompting_0.txt")

PROMPT_3 = construct_CoT_prompt(3)
results_3, accuracy_3 = run_benchmark_test(
    dataset=gsm8k_test,
    prompt=PROMPT_3,
    VERBOSE=False,
    num_samples=50
)
save_final_result(results_3, accuracy_3, "CoT_prompting_3.txt")

PROMPT_5 = construct_CoT_prompt(5)
results_5, accuracy_5 = run_benchmark_test(
    dataset=gsm8k_test,
    prompt=PROMPT_5,
    VERBOSE=False,
    num_samples=50
)
save_final_result(results_5, accuracy_5, "CoT_prompting_5.txt")

 10%|█         | 5/50 [00:12<02:04,  2.78s/it]

Progress: [5/50]
Current Acc.: [80.00%]


 20%|██        | 10/50 [00:24<01:42,  2.55s/it]

Progress: [10/50]
Current Acc.: [70.00%]


 30%|███       | 15/50 [00:36<01:22,  2.36s/it]

...

Progress: [40/50]
Current Acc.: [55.00%]


 90%|█████████ | 45/50 [09:14<01:02, 12.48s/it]

Progress: [45/50]
Current Acc.: [55.56%]


100%|██████████| 50/50 [10:05<00:00, 12.11s/it]

Progress: [50/50]
Current Acc.: [58.00%]


### Construct your prompt!!

목표: 본인만의 프롬프트를 통해 정답률을 더 끌어올려보기!
- gsm8k의 train 데이터셋에서 예시를 가져온 다음 (자유롭게!)
- 그 예시들에 대한 풀이 과정을 만들어주세요!
- 모든 것들이 자유입니다! Direct Prompting, CoT Prompting을 한 결과보다 정답률만 높으면 돼요.

In [ ]:
def generate_response_using_Llama(
        prompt: str,
        model: str = "llama-3.1-8b-instant"
    ):
    try:
        chat_completion = client.chat.completions.create(
            messages=[
                {"role": "system",
                 "content": "Solve math. Final line must be 'Answer: <number>'."},
                {"role": "user", "content": prompt}
            ],
            model=model,
            temperature=0.05,  
            stream=False
        )
        return chat_completion.choices[0].message.content
    
    except Exception as e:
        print(f"API call error: {str(e)}")
        return None

def construct_my_prompt(example_list: List[str], num_examples: int = 3):
    train_dataset = gsm8k_train
 
    good_indices = []
    for i in range(400):
        try:
            answer = train_dataset['answer'][i]
            final_answer = answer.split("####")[-1].strip()
            num = float(final_answer.replace(",", ""))
            
            if num <= 50 or num % 10 == 0:
                good_indices.append(i)
        except:
            continue
    
    if len(good_indices) < num_examples:
        sampled_indices = random.sample(range(len(train_dataset['question'])), num_examples)
    else:
        sampled_indices = random.sample(good_indices, num_examples)
    
    prompt = ""
    
    for idx in sampled_indices:
        question = train_dataset['question'][idx]
        answer = train_dataset['answer'][idx]
        final_answer = answer.split("####")[-1].strip()
        prompt += f"Q: {question}\nAnswer: {final_answer}\n\n"
    
    prompt += (
    "Q: {question}\n"
    "Let's think step by step.\n"
    "Finally, output ONLY the final number after the tag 'Answer:' with no extra text.\n"
    "Answer:"
    )
    
    return prompt

In [ ]:
for shot in [0, 3, 5]:
    print(f"\n===== My Prompting {shot}-shot 시작 =====")
    
    PROMPT = construct_my_prompt([], num_examples=shot)
    results, accuracy = run_benchmark_test(
        dataset=gsm8k_test,
        prompt=PROMPT,
        VERBOSE=False,
        num_samples=50
    )
    save_final_result(results, accuracy, f"My_prompting_{shot}.txt")
    
    print(f"My Prompting {shot}-shot 완료! 정답률: {accuracy:.2f}%")


===== My Prompting 0-shot 시작 =====


 10%|█         | 5/50 [00:02<00:19,  2.28it/s]

Progress: [5/50]
Current Acc.: [80.00%]


 20%|██        | 10/50 [00:04<00:22,  1.79it/s]

Progress: [10/50]
Current Acc.: [80.00%]


...

Progress: [40/50]
Current Acc.: [77.50%]


 90%|█████████ | 45/50 [04:28<00:29,  5.84s/it]

Progress: [45/50]
Current Acc.: [75.56%]


100%|██████████| 50/50 [04:57<00:00,  5.95s/it]

Progress: [50/50]
Current Acc.: [78.00%]
My Prompting 5-shot 완료! 정답률: 0.78%


### 보고서 작성하기
#### 아래의 내용이 포함되면 됩니다!

1. Direct Prompting, CoT Prompting, My Prompting을 0 shot, 3 shot, 5 shot 정답률을 표로 보여주세요!
2. CoT Prompting이 Direct Prompting에 비해 왜 좋을 수 있는지에 대해서 서술해주세요!
3. 본인이 작성한 프롬프트 기법이 CoT에 비해서 왜 더 좋을 수 있는지에 대해서 설명해주세요!
4. 최종적으로, `PROMPTING.md`에 보고서를 작성해주세요!